In [2]:
import os
import re
import upload_v2 as v2

In [3]:
dict_refer = v2.dict_refer
comp = v2.comp

In [3]:
files = ["20495237 보증인 김수희 양도통지서.pdf"]

for f in files:
    print("-----------------------------------------")
    name_items = {"key" :"", "name" :"", "docu" :"", "event" :"", "extra" :"", "date" :""}
    name_extra = None
    n = os.path.splitext(f)[0]
    ext = os.path.splitext(f)[1]

    name_items["key"], n = v2.keyFnc(n)
    print("afterkeyF n = ", n)
    isEvent, docuEventNameextra = v2.eventFnc(n) ### docu, event
    print("isEvent", isEvent)
    if isEvent == 2 :
        print("경정사건인데 nodocu")
        continue
    elif isEvent == 1 : # 정상
        name_items["docu"] = docuEventNameextra[0] #사건번호>>docu 할당 #############
        name_items["event"] = docuEventNameextra[1]
        name_extra = docuEventNameextra[2]
    else : # 사건번호 없음
        isDocu, docuNameExtra = v2.setDocu(n)
        print("isDocu : ", isDocu, "docuNameExtra : ", docuNameExtra)
        if isDocu :
            name_items["docu"] = docuNameExtra[0]#비사건번호>>docu 할당 #############
            name_extra = docuNameExtra[1]
        else : # nodocu
            print("setDocu 결과 nodocu")
            continue
    date, temp = v2.dateFnc(name_extra) ### date, name, extra
    print(f"date : {date}, temp = {temp}")
    name, extra = v2.rmNeedlessSharp(temp)   
    print(f"name = {name}, extra={extra}")
    if len(name) < 2 : 
        try :
            name = dict_refer[name_items["key"]][2]
        except :
            print("이름을 찾는데 키가 없어.")
        
        for ptrn in re.findall("[가-힣a-zA-Z]+", name) : # 새로 가져온 name에만 적용
            extra = re.sub(ptrn+"의?", "", extra) #다행히 ')의'는 '~~(주)의'밖에 안 보이네. 혹시 하다가 예외 많으면 \b의\b를 따로 해주는 것으로
        
        name, extra = v2.rmNeedlessSharp(name+"#"+extra) #list를 반환하기 때문에 
    
    name = v2.rm_s(name)
    extra = v2.rm_s(extra)

    name_items["name"] = name #### name, extra, date 할당 ######################
    name_items["extra"] = extra
    name_items["date"] = date
    new_f = "_".join(filter(lambda x :bool(x), name_items.values())) + ext
    print(f, ">>>", new_f)
    

-----------------------------------------
afterkeyF n =  보증인 김수희 양도통지서
isEvent 0
isDocu :  True docuNameExtra :  ['양도통지서', '보증인 김수희#']
date : , temp = 보증인 김수희#
name = 보증인 김수희, extra=
20495237 보증인 김수희 양도통지서.pdf >>> 20495237_보증인 김수희_양도통지서.pdf


In [4]:
# depth test
files = ["20407898 박한수 양도통지서.pdf"]
for f in files:
    new_f = f 
    depth1, depth2, depth3 = v2.setDepth(new_f)########### depth ###########

    print(depth1, depth2, depth3)

wrongdocu  


In [7]:
p = re.compile(r"기타|집행문\s?부여의\s?소|배송[가-힣]+|(채권|양도|양수|매매).*계약|화해(?!권고)|대위변제|분할|분납|상환|감면|(상속)?\s?한정\s?승인|상속\s?포기|지방세|세목별|과세|.*내용증명|.*답변서|.*진술서|.*보정(서|명령)") # 제거가 아니므로 첫 글자만 잘 찾으면 된다.
p1 = re.compile(r"재산\s?명시|[가-힣\b]*(부?동산)?\s?(가?압류|경매)(?!물건)|[가-힣\b]*동산|[가-힣\b]*추심|[가-힣\b]*유체|[가-힣\b]*배당[가-힣]|.*진술\s?최고서?")#진술최고서는 강제집행, 와 그냥 진술서(noupload)는 다르지만 파일명만으로는 확인할 수 없다.
fs = ["하나로 내용증명", "중앙 농협진술서", "농협은행 3채무자 진술최고서"]
print(*[(f,p.search(f)) for f in fs], sep='\n')
print("===============")
print(*[(f,p1.search(f)) for f in fs], sep='\n')


('하나로 내용증명', <re.Match object; span=(0, 8), match='하나로 내용증명'>)
('중앙 농협진술서', <re.Match object; span=(0, 8), match='중앙 농협진술서'>)
('농협은행 3채무자 진술최고서', None)
('하나로 내용증명', None)
('중앙 농협진술서', None)
('농협은행 3채무자 진술최고서', <re.Match object; span=(0, 15), match='농협은행 3채무자 진술최고서'>)


In [58]:
import re
stem = "(주)동성제약 박현숙(자) 12345"
allName = "(주)동성제약 박현숙(자) "

spanList = []
for ptrn in re.findall("[가-힣a-zA-Z㈜]+|\([가-힣]\)", allName) : #(주)를 안 썼을 수도 있으니 따로 빼준다.
    ptrn = re.sub("\(", "\\(", ptrn) #search에서 ()인식을 위해
    ptrn = re.sub("\)", "\\)", ptrn)
    p = re.search(ptrn, stem)
    print(p)
    if p :
        spanList.append(p.start())
        spanList.append(p.end())
        spanList.sort()
print(spanList)
name = stem[spanList[0]:spanList[-1]]
extra = stem[ : spanList[0]] + stem[spanList[-1] : ] # 괄호때문에 그냥 이렇게 함.


print(name)
print("extra", extra)


<re.Match object; span=(0, 3), match='(주)'>
<re.Match object; span=(3, 7), match='동성제약'>
<re.Match object; span=(8, 11), match='박현숙'>
<re.Match object; span=(11, 14), match='(자)'>
[0, 3, 3, 7, 8, 11, 11, 14]
(주)동성제약 박현숙(자)
extra  12345


In [42]:
allName = "(주)동성제약 박현숙(자) "
allName.find()